# Test `caterva_get_slice()`

In [2]:
import pycblosc2 as pyc
import pcatervaW as cat
import numpy as np

## Define parameters

In [4]:
shape = [512, 512, 512, 1, 1, 1, 1, 1]
cshape = [64, 32, 64, 1, 1, 1, 1, 1]

start = [130, 101, 230, 0, 0, 0, 0, 0]
stop = [220, 300, 300, 1, 1, 1, 1, 1]
step = [1, 1, 1, 1, 1, 1, 1, 1]

dim = 3

slice_np = tuple([slice(i, j) for i, j in zip(reversed(start[0:dim]), reversed(stop[0:dim]))])

shape_slice = [stop[i] - start[i] for i in range(8)]
cshape_slice = [32, 32, 32, 1, 1, 1, 1, 1]


size = np.prod(shape)
csize = np.prod(cshape)

size_slice = np.prod(shape_slice)
csize_slice = np.prod(cshape_slice)

dtype = np.float64
typesize = np.dtype(dtype).itemsize

## Create params and caterva containers

In [5]:
cp = cat.caterva_create_cparams(compcode=1, clevel=5, use_dict=0, typesize=typesize, nthreads=4,
                               blocksize=1024, schunk=None, filters=[0, 0, 0, 0, 1],
                               filters_meta=[0, 0, 0, 0, 2])

dp = cat.caterva_create_dparams(nthreads=1, schunk=None)

pp = cat.caterva_create_pparams(shape,cshape, dim)

pp_slice = cat.caterva_create_pparams(shape_slice, cshape_slice, dim)

carr = cat.caterva_new_array(cp, dp, pp)
carr_slice = cat.caterva_new_array(cp, dp, pp_slice)

## Create example data

In [6]:
arr = np.linspace(0, size - 1, size, dtype=dtype).reshape([i for i in reversed(shape[0:dim])])
arr_slice = np.empty(size_slice, dtype=dtype).reshape([i for i in reversed(shape_slice[0:dim])])

## Get a slice using caterva

In [7]:
error = cat.caterva_schunk_fill_from_array(arr, carr)

error = cat.caterva_get_slice(carr, carr_slice, start, stop, step)

error = cat.caterva_array_fill_from_schunk(carr_slice, arr_slice)

## Get a slice using *numpy*

In [8]:
arr_np = arr[slice_np]

## Test results

In [9]:
np.testing.assert_array_almost_equal(arr_slice, arr_np)